In [13]:
import tensorflow as tf
import numpy as np

In [2]:
INTENT_LABELS = [
    'Book',
    'CheckStatus',
    'GoToHell',
    'OMG',
    'Other'
]
NUM_INTENTS = len(INTENT_LABELS)

In [3]:
def model_fn(features, labels, mode):
    tokens = features['tokens']
    token_lengths = features['token_lengths']
    intent_logits = tf.random_uniform((tf.shape(tokens)[0], NUM_INTENTS), maxval=10.)
    intent_proba = tf.nn.softmax(intent_logits)
    intent_i = tf.argmax(intent_proba, axis=-1)
    intent_confidence = tf.reduce_max(intent_proba, axis=-1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        label_lookup = tf.contrib.lookup.index_to_string_table_from_tensor(INTENT_LABELS, 
                                                                           default_value=INTENT_LABELS[-1])
        return tf.estimator.EstimatorSpec(mode, predictions={
            'intent_i' : intent_i,
            'confidence' : intent_confidence,
            'input_length' : token_lengths,
            'intent' : label_lookup.lookup(intent_i)
        })
    elif mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_or_create_global_step()
        train_op = tf.assign_add(global_step, 1)
        return tf.estimator.EstimatorSpec(mode, loss=tf.constant(0), train_op=train_op)
    else:
        raise NotImplementedError()

In [4]:
def fake_input_fn():
    return ({
        'tokens' : tf.constant([['Я', 'хочу', 'поссать']]),
        'token_lengths' : tf.constant([3])
    }, None)

In [5]:
def serving_fn():
    # we expect a single message only (no batches) from an upstream model
    receiver_msg_ph = tf.placeholder(tf.string, (None,))
    return tf.estimator.export.ServingInputReceiver(features = {
        # but the model fn works with batches
        'tokens' : tf.expand_dims(receiver_msg_ph, axis=0),
        'token_lengths' : tf.expand_dims(tf.shape(receiver_msg_ph)[0], axis=0)
    }, receiver_tensors = {
        'preprocessed_msg' : receiver_msg_ph
    })

In [6]:
!rm -rf wrk/baseline-v0

In [7]:
estimator = tf.estimator.Estimator(model_fn, model_dir='wrk/baseline-v0')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'wrk/baseline-v0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa54fd18438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
estimator.train(fake_input_fn, max_steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into wrk/baseline-v0/model.ckpt.
INFO:tensorflow:loss = 0, step = 1
INFO:tensorflow:Saving checkpoints for 1 into wrk/baseline-v0/model.ckpt.
INFO:tensorflow:Loss for final step: 0.


In [17]:
[p for p in estimator.predict(tf.estimator.inputs.numpy_input_fn({
    'tokens' : np.array([['Я', 'хочц', 'поссикать', 'gfdgf']]),
    'token_lengths' : np.array([3])
}, shuffle=False))]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from wrk/baseline-v0/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'intent_i': 2,
  'confidence': 0.5903566,
  'input_length': 3,
  'intent': b'GoToHell'}]

In [16]:
estimator.export_savedmodel('saved-models', serving_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from wrk/baseline-v0/model.ckpt-1
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved-models/temp-b'1545335768'/saved_model.pb


b'saved-models/1545335768'